In [ ]:
from ipywidgets import IntSlider, VBox, HBox, Dropdown, Output, IntText, Layout, Widget, Button, HTML
from IPython.display import display, clear_output
from colors import Colors
from state import State, get_event_info
from string_utils import Dashboard, print_resources, pretty_print_number, pretty_print_time, print_plan
from time import sleep
from strategy import Strategy
from threading import Thread

#hack to prevent output widget from becoming a scrolling output
display(HTML("""
    <style>
       .jupyter-widgets-output-area .output_scroll {
            height: 100% !important;
            border-radius: unset !important;
            -webkit-box-shadow: unset !important;
            box-shadow: unset !important;
        }
    </style>
"""))

MAX_LEVEL = 10
MAX_CHAMPIONS = 8
MAX_RESOURCES = 4

class ClearOutput(Output):
    """
    The context manager for output widgets can be a bit wonky.
    This extension provides a clean mechanism for replicating
    the following pattern:
    
    with out:
        clear_output(wait = True)
        print(text)
        
    which appears to be thread-safe
    """
    
    def clear_print(self, text):
        self.outputs = ({'output_type': 'stream', 'name': 'stdout', 'text': text},)

class NoWith:
    """
    I don't know... trying to create a with block that does nothing special
    """
    __enter__ = lambda *args: None
    __exit__ = lambda *args: None

class Solver(Thread):
    def __init__(self, strategy, context=None, *args, **kwargs):
        """
        strategy is a Strategy object that will be solved
        context is the output context to be used, if None it will use stdout
        the dashboard object will print to the output widget if supplied as context
        """
        super().__init__()
        self.strategy = strategy
        self.strategy.stop = False
        self.dashboard = Dashboard()
        self.dashboard.set_state(self.strategy.state)
        if context is not None:
            self.dashboard.set_context(context)
            self.context = context
        self.args = args
        self.kwargs = kwargs
        
    def interrupt(self):
        """
        Call this to set a flag to stop the program
        """
        self.strategy.stop = True
        
    def run(self):
        self.strategy.plan(
            dashboard = self.dashboard, 
            mode = 'hybrid', 
            swap_levels = 1,
            *self.args, 
            **self.kwargs
        )
        self.dashboard.show(force = True, include_urgency = True)
        
        
class Time(HBox):
    def __init__(self):
        self.display = ClearOutput(layout = Layout(width = '20em'))
        style = {'description_width' : '6em'}
        layout = Layout(width = 'max-content')
        self.time = 3*24*3600
        self.days = Dropdown(description = 'days', options = range(4), index = 3, style = style, layout = layout)
        self.hours = Dropdown(description = 'hours', options = range(24), index = 0, style = style, layout = layout)
        self.minutes = Dropdown(description='minutes', options = range(60), index = 0, style = style, layout = layout)
        self.seconds = Dropdown(description='seconds', options = range(60), index = 0, style = style, layout = layout)
        super().__init__([self.display, self.days, self.hours, self.minutes, self.seconds])
        for widget in [self.days, self.hours, self.minutes, self.seconds]:
            widget.observe(self.update)
        self.update()
        
    def update(self, event = None):
        self.time = (
            self.days.value * 24 * 3600 + 
            self.hours.value * 3600 + 
            self.minutes.value * 60 + 
            self.seconds.value
        )
        self.display.clear_print(pretty_print_time(self.time))
        
        
        
class Resource (VBox):
    def __init__(self, color, *args, **kwargs):
        self.name = ClearOutput()
        self.resource = IntText(continuous_update = False, layout = Layout(width = 'auto'))
        self.resource.observe(self.on_resource_change)
        self.pretty = ClearOutput()
        self.color = color
        super().__init__([self.name, self.resource, self.pretty], *args, **kwargs)
        
    def on_resource_change(self, change):
        self.pretty.clear_print(Colors.color(pretty_print_number(self.resource.value), self.color))
        

class Resources(HBox):
    COLORS = ['blue', 'green', 'red', 'gold', 'bold']
    def __init__(self):
        layout = Layout(width = '100%')
        event_info = get_event_info()
        self.resources = []
        for i in range(5):
            self.resources.append(Resource(self.COLORS[i], layout = layout))
        super().__init__(self.resources)
                
class Upgrade(HBox):
    def __init__(self, info):
        style = {'description_width' : '12em'}
        layout = Layout(width = '36em')
        self.slider = IntSlider(value = 0, min = 0, max = MAX_LEVEL, style = style, layout = layout)
        self.slider.observe(self.update)
        self.cost = ClearOutput()
        self.info = info
        self.callback_f = lambda : None
        super().__init__([self.slider, self.cost])
        self.update()
        
    def set_callback(self, f):
        self.callback_f = f
        
    def callback(self):
        self.callback_f()
        #allow higher level structure to receive updates from self
        
    def update(self, event = None):
        self.slider.description = self.info['name']
        self.slider.max = self.info['max_level']
        
        if self.slider.value == self.info['max_level']:
            self.cost.clear_print(Colors.color('MAXED', 'rainbow'))
        else:
            self.cost.clear_print(print_resources(self.info['upgrade_costs'][self.slider.value]))
        self.callback()
        
class Gem(Upgrade):
    def __init__(self):
        super().__init__(None)
        
    def update(self, event = None):
        cost = [100, 250, 500, 750, 1000] #i think? don't really care
        self.slider.description = "gem production boost"
        self.slider.max = 5
        if self.slider.value == 5:
            self.cost.clear_print(Colors.color('MAXED', 'rainbow'))
        else:
            self.cost.clear_print(Colors.color(f"{cost[self.slider.value]} gems", 'rainbow'))
        self.callback()

class RMD(Upgrade):
    def __init__(self):
        super().__init__(None)
        
    def update(self, event = None):
        cost = [0] #i think? don't really care
        self.slider.description = "real money boost"
        self.slider.max = 1
        if self.slider.value == 1:
            self.cost.clear_print(Colors.color('MAXED', 'rainbow'))
        else:
            self.cost.clear_print(Colors.color(f"$$$$$", 'rainbow'))
        self.callback()
                 
    
class GUI(VBox):
    EVENTS = ['other_tower', 'sid_in_space', 'happy_time', 'aqua_adventure', 'lost_in_time', 'through_the_portal']
    def __init__(self):
        self.thread = None
        self.state = State()
        self.dropdown = Dropdown(options = self.EVENTS, index = 0, description = 'event')
        self.dropdown.observe(self.change_event)
        self.create_plan_button = Button(description = 'create_plan')
        self.create_plan_button.on_click(self.execute)
        self.interrupt_button = Button(description = 'interrupt')
        self.interrupt_button.on_click(self.interrupt)
        #create custom thread object with interrupt
        self.strategy_output = ClearOutput(layout = Layout(height = '5000px'))
        super().__init__([])
        self.build()
        
    def change_event(self, event):
        self.state.set_event_by_name(self.dropdown.value)
        self.build()
    
    def sync(self):
        resources = ['blue', 'green', 'red', 'gold', 'damage']
        event_info = get_event_info()
        self.state.time = self.time.time
        self.state.gem_level = self.gem_level.slider.value
        self.state.boost_level = self.boost_level.slider.value
        self.state.damage_level = self.damage.slider.value
        self.state.speed_level = self.speed.slider.value
        if event_info['has_speed2']:
            self.state.speed2_level = self.speed2.slider.value
            
        #setting structs and arrays of a non cimported class is tricky
        #need to set them atomically, and not by setting their components
        self.state.resources = {
            resources[i] : self.current_resources.resources[i].resource.value for i in range(5)
        }
        self.state.champion_levels = [
            self.champions[i].slider.value for i in range(event_info['n_champions'])
        ] + [0] * (MAX_CHAMPIONS - event_info['n_champions'])
        self.state.resource_levels = [
            self.resources[i].slider.value for i in range(event_info['n_resources'])
        ] + [0] * (MAX_RESOURCES - event_info['n_resources'])
        self.state.update_resources_per_second()
        
    def interrupt(self, event = None):
        if self.thread is not None:
            self.thread.interrupt()
            self.thread = None
    
    def execute(self, event = None):
        self.sync()
        if self.thread is not None:
            self.thread.interrupt()
            
        self.strategy = Strategy(state = self.state)
        self.thread = Solver(self.strategy, self.strategy_output)
        self.thread.start()
        
    def build(self):
        """called on init or when the event is changed"""
        # these names do not appear in the event_info currently
        DAMAGE_NAME = ['Damage', 'Distance', 'Hearts', 'Depth', 'Time Dust', 'Damage']
        self.interrupt()
        event_info = get_event_info()
        self.current_resources = Resources()
        for i in range(event_info['n_resources']):
            self.current_resources.resources[i].name.clear_print(
                Colors.color(event_info['resources'][i]['name'].decode(), self.current_resources.COLORS[i])
            )
            self.current_resources.resources[i].layout = Layout(width = '100%', display = 'block')
        for j in range(i+1, 4):
            self.current_resources.resources[j].layout = Layout(display = 'none')
        self.current_resources.resources[-1].name.clear_print(Colors.color(DAMAGE_NAME[self.dropdown.index], 'bold'))
        self.time = Time()
        self.gem_level = Gem()
        self.boost_level = RMD()
        self.champions = [Upgrade(event_info['champions'][i]) for i in range(event_info['n_champions'])]
        self.resources = [Upgrade(event_info['resources'][i]) for i in range(event_info['n_resources'])]
        self.damage = Upgrade(event_info['damage'])
        self.speed = Upgrade(event_info['speed'])
        self.speed2 = Upgrade(event_info['speed2'])if event_info['has_speed2'] else HBox()
        self.children = (
            self.dropdown,
            self.time,
            self.current_resources, 
            *self.champions,
            self.gem_level,
            self.boost_level,
            *self.resources,
            self.speed,
            self.damage,
            self.speed2,
            HBox([self.create_plan_button, self.interrupt_button]),
            self.strategy_output
        )
        
State.set_event_by_name('other_tower')

gui = GUI()
gui